## Approach (inspired by Jurafsky et al. 2009):

1. Create seed pattern for relationships (_child-of_, _parent-of_)
2. Run NER-Tagger on large text
3. Extract training sentences
4. Create feature vectors
5. Train model


In [28]:
import re

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.tag import StanfordNERTagger

model = '../models/dewac_175m_600.crf.ser.gz'
#model = '../models/hgc_175m_600.crf.ser.gz'
#model = '../models/german.conll.germeval2014.hgc_175m_600.crf.ser.gz'

st = StanfordNERTagger(model,
                       '../models/stanford-ner.jar',
                       encoding='utf-8')

c:\program files\python\lib\site-packages\nltk\tag\stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [1]:
relationship_list = ['vater', 'mutter', 'sohn', 'tochter', 'bruder', 'schwester', 'enkel', 'enkelin', 'nichte', 
                     'neffe', 'onkel', 'tante']

In [2]:
me_list = ['ich', 'meine', 'mein']

In [11]:
relationship_tuples = [('REL', 'PME', 'PER'), ('REL', 'PER', 'PME'), ('REL', 'PER', 'PER')]

In [32]:
training_text = u'''Meine Enkelin Lisa und mein Enkel Lukas fliegen morgen nach London. Sie sind zum ersten Mal in England. 
Meine Tochter Christina und mein Sohn Luis fahren nach Berlin. Ich bin geboren zu York im Jahre 1632, als Kind angesehener Leute, die ursprünglich nicht aus jener Gegend stammten. 
Mein Vater, ein Ausländer, aus Bremen gebürtig, hatte sich zuerst in Hull niedergelassen, war dort als Kaufmann zu 
hübschem Vermögen gekommen und dann, nachdem er sein Geschäft aufgegeben hatte, nach York gezogen. 
Hier heiratete er meine Mutter, eine geborene Robinson.
Ich hatte zwei ältere Brüder. Der eine von ihnen, welcher als Oberstleutnant bei einem englischen, 
früher von dem berühmten Oberst Lockhart befehligten Infanterieregiment in Flandern diente, 
fiel in der Schlacht bei Dünkirchen. Was aus dem jüngeren geworden ist, habe ich ebenso wenig in Erfahrung bringen können, 
als meine Eltern je Kenntnisse von meinen eignen Schicksalen erhalten haben.'''

#### Tag named entities using Stanford NER and manually created list
Entity types:
* PER: Person (I-PER) recognized by Stanford NER tagger
* REL: Relation some kinds of German named relationhips following the manually created *relationship_list*
* PME: 'Person-Me'; the user mentioning him-/herself as specified in the *me_list*

#### Add training instances to the corresponding relationship tuples

In [36]:
training_dict = {}

for sentence in sent_tokenize(training_text):
    ner_tuples = st.tag(sentence.split())
    rel_tuples = ()
    
    for ner_tuple in ner_tuples:        
        if 'I-PER' in ner_tuple:
            rel_tuples += ('PER',)
        elif ner_tuple[0].lower() in relationship_list:
            rel_tuples += ('REL',)
        elif ner_tuple[0].lower() in me_list:
            rel_tuples += ('PME',)
    
    if len(rel_tuples) >= 2:
        clean_sentence = re.sub(r'\W', ' ', sentence) # remove non-word characters
        clean_sentence = re.sub(r'\s{2,}', ' ', clean_sentence)  # remove two or more consecutive whitespaces
        if rel_tuples in training_dict.keys():
            curr_values = training_dict[rel_tuples]
            curr_values.append(clean_sentence)
        else:
            curr_values = [clean_sentence]
            
        training_dict[rel_tuples] = curr_values
        
training_dict

{('PME',
  'REL',
  'PER',
  'PME',
  'REL',
  'PER'): ['Meine Enkelin Lisa und mein Enkel Lukas fliegen morgen nach London ', 'Meine Tochter Christina und mein Sohn Luis fahren nach Berlin '],
 ('PME',
  'PME'): ['Was aus dem jüngeren geworden ist habe ich ebenso wenig in Erfahrung bringen können als meine Eltern je Kenntnisse von meinen eignen Schicksalen erhalten haben ']}

### Extract training features (lexical and syntactic)
* sequence of words between two entities
* POS-Tags of theses words
* Flag indicating which entity came first in sentence
* k-words left of entity 1 (M1)
* k-words right of entity 2 (M2)

In [38]:
class Feature:
    def __init__(M1, M2, nextword, path):
        this.M1 = M1
        this.M2 = M2
        this.nextword = nextword
        this.path = path        


In [ ]:
features = []

for key, value in relationship_dict.items():
     
    